<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2019</h1>
<h1 style="font-size: 250%;">Problem Set #3</h1>
<h1 style="font-size: 200%;">Version 1.1</h1>
<h1>Issued Tuesday 9/22/2020; Due Thursday 10/08/2020</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be accepted, but your lowest problem-set grade will be dropped. Feel free to get help from others, but the work you submit in should be your own.

In [31]:
# Please fill out the entries in this Cell.
# By sigining it you are agreeing to the UNC honor code
class Student:
    first = "Nelson"
    last = "Lopez"
    onyen = "nelson19"
    pid = "730157511"

In the following questions you will need to write Python3 programs to perform queries
    on and modify the <a href="http://csbio.unc.edu/mcmillan/Media/NCCOVID19.db" download>NCCOVID19.db</a>
    database used in Problem Set #2. **You are only allowed to import the standard "sqlite3"
    and the "time" Python packages.**

---
**Problem 1:** The file <a href="http://csbio.unc.edu/mcmillan/Media/NC2018Mortality.csv" download="NC2018Mortality.csv">NC2018Mortality.csv</a> contains a list of deaths by race and sex in each of North Carolina's counties. Write a Python3 program that creates a new table in the NCCOVID19.db database and then loads that table into the database using values from this file. The table should have the following schema:

<table style="border-collapse: collapse; border: 1px solid black;"><tbody>
<tr style="border-collapse: collapse; border: 1px solid black;">
    <th colspan="7" style="text-align:center; background-color: skyblue;">Mortality</th></tr>
<tr style="border-collapse: collapse; border: 1px solid black;">
    <td style="border-collapse: collapse; border: 1px solid black;"><u>fips</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>year</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>race</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;"><u>sex</u></td>
    <td style="border-collapse: collapse; border: 1px solid black;">count</td>
</tr>
</tbody></table>

Do not parse the file using pandas or Python's built-in "csv" modules "reader" method. This will require that you open the file, read its contents, split them into lines, extract the header line,  

In [7]:
import sqlite3

db = sqlite3.connect('NCCOVID19.db')
cursor = db.cursor()

sql1='''CREATE TABLE IF NOT EXISTS Mortality (
    fips INTEGER,
    year INTEGER,
    race TEXT,
    sex TEXT,
    count INTEGER,
    PRIMARY KEY (fips, year, race, sex),
    FOREIGN KEY(fips) REFERENCES County(fips))'''

cursor.execute(sql1)

fd = open('NC2018Mortality.csv', 'r')
s = fd.read()
s = s.split('\n')

first = 0;
for county in s:
    if county == "County,white-male,white-female,black-male,black-female,asian-male,asian-female,aian-male,aian-female,other-male,other-female":
        continue
    if county == "":
        continue

    x = county.split(',')
    countyName = x[0].upper()
    
    
    sql2 = '''SELECT County.fips FROM County WHERE UPPER(County.name) = ?'''
    cursor.execute(sql2,(x[0].upper(),))
    countyfips = int(str((cursor.fetchall()))[2:7])
    year = 2018

    count = 1
    for pop in x:
        try:
            pop = int(pop)
        except:
            continue
        popcount = pop
        if count == 1: #White Male
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "white", "male", popcount))
        if count == 2: #White Female
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "white", "female", popcount))
        if count == 3: #Black Male
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "black", "male", popcount))
        if count == 4: #Black Female
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "black", "female", popcount))
        if count == 5: #Asian Male
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "asian", "male", popcount))
        if count == 6: #Asian Female
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "asian", "female", popcount))
        if count == 7: #aian Male
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "aian", "male", popcount))
        if count == 8: #aian Female
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "aian", "female", popcount))
        if count == 9: #other Male
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "other", "male", popcount))
        if count == 10: #other Female
            sql = '''INSERT OR IGNORE INTO Mortality (fips, year, race, sex, count)
            VALUES (?, ?, ?, ?, ?)'''
            cursor.execute(sql, (countyfips, year, "other", "female", popcount))
        count+=1
        db.commit()
    
    
db.close()

---
**Problem 2:** Use the new table that you created in **Problem 1** to create the following Python3 function that estimates the number of 2018 deaths in each demographic category for a given county. These estimates are based on the assumption that *the number of mortality deaths in each demographic's age range is proportional to the 2018 population fraction in that same demogrphic's age range*. Your function should return a list of <code>(race,sex,agelo,agehi,count)</code> tuples, where the sum of all counts should match the total number of 2018 deaths in that county. All <code>counts</code> should be integers. All of your code must be within the body of the given function.

In [63]:
import sqlite3


def covidDemographicsEstimate(countyName):
    demographicGroup = []
    db = sqlite3.connect("NCCOVID19.db")
    db.row_factory = sqlite3.Row
    db.text_factory = str
    cursor = db.cursor()
    # Enter your Python3 code here;
    sql = '''SELECT DISTINCT County.fips FROM County WHERE County.name = ?'''
    cursor.execute(sql, (countyName,))
    fips = cursor.fetchone()
    fips = fips[0]

    sql = '''SELECT Demographics.race, Demographics.sex, Demographics.agelo, Demographics.agehi, Demographics.count FROM Demographics WHERE Demographics.fips = ? AND Demographics.year = 2018'''
    cursor.execute(sql, (fips,))
    demo = cursor.fetchall()

    

    count = 0;
    for pop in demo:
        insertion = []
        insertion.append(pop['race']) 
        insertion.append(pop['sex'])
        insertion.append(pop['agelo'])
        insertion.append(pop['agehi'])
        #insertion.append(pop['count'])
        
        sql = '''SELECT SUM(Demographics.count) FROM Demographics WHERE Demographics.fips = ? AND Demographics.year = 2018 AND Demographics.race = ? AND Demographics.sex = ? GROUP BY Demographics.fips'''
        cursor.execute(sql, (fips, pop['race'],pop['sex'],))
        totalRaceSexPop = cursor.fetchone()
        
       #insertion.append(totalPopulation[0])\
        #print(pop['count'])
        #print(totalRaceSexPop[0])
        proportion = (pop['count'])/totalRaceSexPop[0]
        #Need to Fix Ratio and figure out what I'm  returning 
        #insertion.append(ratio)

        sql = '''SELECT Mortality.fips, SUM(Mortality.count) AS count FROM Mortality WHERE Mortality.fips = ? AND Mortality.year = 2018 AND Mortality.race = ? AND Mortality.sex = ?'''
        cursor.execute(sql, (fips, pop['race'], pop['sex'],))
        mortCount = cursor.fetchall()
        for obj in mortCount:
            assignedDeaths = round((obj['count']*proportion))
            insertion.append(assignedDeaths)


        demographicGroup.append(insertion)
        count+=1
    # End of my code
    return demographicGroup

32224


[['aian', 'female', 0, 2, 0],
 ['aian', 'female', 3, 4, 0],
 ['aian', 'female', 5, 5, 0],
 ['aian', 'female', 6, 9, 0],
 ['aian', 'female', 10, 13, 0],
 ['aian', 'female', 14, 14, 0],
 ['aian', 'female', 15, 15, 0],
 ['aian', 'female', 16, 17, 0],
 ['aian', 'female', 18, 19, 0],
 ['aian', 'female', 20, 24, 0],
 ['aian', 'female', 25, 34, 0],
 ['aian', 'female', 35, 44, 0],
 ['aian', 'female', 45, 54, 0],
 ['aian', 'female', 55, 59, 0],
 ['aian', 'female', 60, 64, 0],
 ['aian', 'female', 65, 74, 0],
 ['aian', 'female', 75, 84, 0],
 ['aian', 'female', 85, 99, 0],
 ['aian', 'female', 100, 200, 0],
 ['aian', 'male', 0, 2, 0],
 ['aian', 'male', 3, 4, 0],
 ['aian', 'male', 5, 5, 0],
 ['aian', 'male', 6, 9, 0],
 ['aian', 'male', 10, 13, 0],
 ['aian', 'male', 14, 14, 0],
 ['aian', 'male', 15, 15, 0],
 ['aian', 'male', 16, 17, 0],
 ['aian', 'male', 18, 19, 0],
 ['aian', 'male', 20, 24, 0],
 ['aian', 'male', 25, 34, 0],
 ['aian', 'male', 35, 44, 0],
 ['aian', 'male', 45, 54, 0],
 ['aian', 'male'

---
**Problem 3:** In the space below write a SQL query that generates a list of (countyName, covidDeaths, totalDeaths2018, percentage) for all counties for which the 2020 Covid-19 deaths exceed 5% of the total deaths for that same county in 2018.

---
**Problem 4:** In the space below wirte a Python3 function that returns a list of all dates for which the given "fips" number accounts for at least the given "percent" of the total covid cases on that date. As an example, the NCCOVID19.db database includes Covid19 records that are not associated with any county. These records have a fips of 0. When your function is called with <code>AtLeastPercent(0, 10.0)</code> it should return the following list of dates:

     ['2020-06-05', '2020-06-10', '2020-06-15', '2020-08-12']

In [44]:
import sqlite3

def AtLeastPercent(fips, percent):
    dateList = []
    db = sqlite3.connect("NCCOVID19.db")
    db.row_factory = sqlite3.Row
    db.text_factory = str
    cursor = db.cursor()
    # Enter your Python3 code here
    
    sql = '''SELECT Covid19.fips, Covid19.date, SUM(Covid19.cases) AS cases FROM Covid19 WHERE Covid19.fips = ? GROUP BY Covid19.date'''
    cursor.execute(sql, (fips,))
    givenCountyCases = cursor.fetchall()

    sql = '''SELECT Covid19.date, SUM(Covid19.cases) AS cases FROM Covid19 GROUP BY Covid19.date'''
    cursor.execute(sql)
    totalStateCases = cursor.fetchall()

    #count = 0;
    #for statecase in totalStateCases:
        #if statecase['date'] == givenCountyCases[count]['date']:
            #dailyPercentage = givenCountyCases[count]['cases']/stateCase['cases']
            #if dailyPercentage >= percent:
                #dateList.append(givenCountyCases[count]['date'])
        #if count == len(givenCountyCases):
            #count = 0
        #count+=1

    for i in range(len(totalStateCases)):
        for j in range(len(givenCountyCases)):
            if totalStateCases[i]['date'] == givenCountyCases[j]['date']:
                dailyPercentage = givenCountyCases[j]['cases']/ totalStateCases[i]['cases']
                dailyPercentage = (dailyPercentage*100.0)
                if dailyPercentage >= percent:
                    dateList.append(givenCountyCases[j]['date'])
        
    # End of Q4 Code
    #for date in dateList:
       # print(date)

    return dateList



['2020-06-05', '2020-06-10', '2020-06-15', '2020-08-12']

---
**Problem 5:** The following query was designed to identify dates where two Counties reported the same number of cases on the same date:<br>
        <pre>
        SELECT C1.name, C2.name, V1.date, V1.cases
        FROM County C1, Covid19 V1, County C2, Covid19 V2
        WHERE C1.fips=V1.fips AND C2.fips=V2.fips
          AND V1.cases=V2.cases AND V1.date=V2.date AND V1.fips < V2.fips AND V1.cases >= 1
        </pre>
        
This query is slow using the default NCCOVID19.db. Write a Python3 program to execute and time
this query along with time to save all rows that it returns into a list for subsequent processing.
You should verify that your list actually extracts data from the rows. In the cell provided
below give the number of rows returned and the time required (no code please).

---
**Problem 6:** Next create an index to improve the performance of this query. In the cell provided below give
the "CREATE INDEX" statement that you found provides the best performance improvement. Also include
the time required for the query after creating the index you chose.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set.

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. 